The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.*

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter.

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
#!/bin/bash

#1a
mkdir -p -v $HOME/students
wget -nc https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
FILE=LCP_22-23_students.csv
if [ ! -f "${HOME}/students/${FILE}" ]; then
    cp $FILE $HOME/students/
    echo "file copied to /home/students"
else
    echo "file already present"
fi
cd $HOME/students

#1b
grep "PoD" $FILE > PoD_students.csv
grep "Physics" $FILE > Phys_students.csv

#1c
COUNT=0
LETTER=A
> initials.txt #clear the file
for x in {A..Z}; do
    TMP=$(grep -c "^${x}" $FILE) #count letter $x
    echo "Letter ${x}, count ${TMP}" >> initials.txt
    if [ $TMP -gt $COUNT ]; then #update if count is bigger than max
        COUNT=$TMP
        LETTER=$x
    fi
done

#1d
echo "Letter ${LETTER}, count ${COUNT}"

#1e
i=0
cat $FILE | while read line
do
    mod=$((i%18))
    if [ $i -gt 0 ]; then
        echo $line >> "group${mod}.csv"
    fi
    i=$((i+1))
done
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
#!/bin/bash

#2a
cd $HOME/students
sed -e "s/,/\t/g" -e "/#/d" data.csv > data.txt
#replace all instances ->   s/find/replace/g
#to delete all instances don't write the replace
#delete all lines containing instance ->    /find/d

#2b
#without using awk, read every line and save it as an array
#then increment the count each time the word ends in an even digit
count=0
input=data.txt
while read -r line; do
    words=($line)
    for word in "${words[@]}"; do
        if [[ "$word" =~ [02468]$ ]]; then
            ((count++))
        fi
    done
done < "$input"
echo $count

#from now on we'll only use awk
#2c
count=0
#awk automatically iterates through every file line, the ith word of each line
#is represented by a variable named i
above=$(awk '{ if ($0*$0+$1*$1+$2*$2>7500) count++} END { print count }' "$input")
count=0
below=$(awk '{ if ($0*$0+$1*$1+$2*$2<7500) count++} END { print count }' "$input")
echo "above:${above}, below:${below}"
#as you see it is much more compact

#2d
read -p "Type the amount of copies" N
for (( j=1; j<=$N; j++ ))
do
    output="data${j}.txt"
    #you need to pass shell variables to awk explicitly
    awk -v var="$j" '{ for (i=1; i<=NF; i++) $i=$i/var } 1' "$input" > "$output"
done
```